In [1]:
# Using test environment unless noted otherwise

import sys
sys.path.append('..') 

import numpy as np

from plaingaussian.normal import N
from plaingaussian.func import logp, dlogp, d2logp, fisher, cholesky_inv
from reffunc import logp as logp_
from reffunc import dlogp_eigh as dlogp_
from reffunc import d2logp as d2logp_
from reffunc import fisher as fisher_

# Matrix inversion

In [13]:
sz = 200

m = np.random.rand(sz, sz)
msq = m @ m.T

print(np.abs(np.linalg.inv(msq) @ msq - np.eye(sz)).max())
print(np.abs(cholesky_inv(msq) @ msq - np.eye(sz)).max())
print("")

%timeit np.linalg.inv(msq)
%timeit cholesky_inv(msq)

6.112573877326881e-11
4.3283250967607065e-11

778 µs ± 88.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
613 µs ± 38.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# logp

In [2]:
sz = 200

m = np.random.rand(sz, sz)
msq = m @ m.T

v = np.random.rand(sz)
v1 = np.random.rand(sz)

xi = N(v1, msq)

print("--- pure time ---")
%timeit logp(v, v1, msq)
%timeit logp_(v, v1, msq)
print("--- wall-clock time ---")
%timeit xi.logp(v)
%timeit logp(v, v1, m @ m.T)
%timeit logp_(v, v1, m @ m.T)

--- pure time ---
275 µs ± 33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
297 µs ± 78.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
--- wall-clock time ---
465 µs ± 41.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
500 µs ± 47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
480 µs ± 51.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
sz = 1000

m = np.random.rand(sz, sz)
msq = m @ m.T

v = np.random.rand(sz)
v1 = np.random.rand(sz)

xi = N(v1, msq)

print("--- pure time ---")
%timeit logp(v, v1, msq)
%timeit logp_(v, v1, msq)
print("--- wall-clock time ---")
%timeit xi.logp(v)
%timeit logp(v, v1, m @ m.T)
%timeit logp_(v, v1, m @ m.T)

--- pure time ---
14.4 ms ± 3.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
14.6 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
--- wall-clock time ---
34.7 ms ± 2.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
34.8 ms ± 2.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
34.8 ms ± 2.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
# Batch operation

sz = 200

m = np.random.rand(sz, sz)
msq = m @ m.T
v1 = np.random.rand(sz)

xi = N(v1, msq)

yy = xi.sample(400)

%timeit xi.logp(yy[0])
%timeit xi.logp(yy[:10])
%timeit xi.logp(yy[:100])
%timeit xi.logp(yy)

460 µs ± 33.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
618 µs ± 102 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
735 µs ± 35.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.33 ms ± 445 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# dlogp

In [24]:
sz = 200
npar = 4

m = np.random.rand(sz, sz)
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz)
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)

%timeit dlogp(v, v1, msq, v2, msq2)
%timeit dlogp_(v, v1, msq, v2, msq2)

888 µs ± 84.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.39 ms ± 390 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
# Single large matrix
sz = 400
npar = 1

m = np.random.rand(sz, sz)
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz)
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)

%timeit dlogp(v, v1, msq, v2, msq2)
%timeit dlogp_(v, v1, msq, v2, msq2)

6.69 ms ± 299 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
26.6 ms ± 2.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
sz = 200
npar = 10

m = np.random.rand(sz, sz)
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz)
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)

%timeit dlogp(v, v1, msq, v2, msq2)
%timeit dlogp_(v, v1, msq, v2, msq2)

1.43 ms ± 95.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
11.1 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
sz = 20
npar = 10

m = np.random.rand(sz, sz)
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz)
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)

%timeit dlogp(v, v1, msq, v2, msq2)
%timeit dlogp_(v, v1, msq, v2, msq2)

60.3 µs ± 2.66 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
155 µs ± 4.54 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# d2logp

In [2]:
sz = 200
npar = 4

m = np.random.rand(sz, sz) 
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz) 
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

mat3 = np.random.rand(npar, npar, sz, sz) 
msq3 = np.einsum('ijkl, ijrl -> ijkr', mat3, mat3)
msq3 = msq3.transpose(1, 0, 2, 3) + msq3

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)
v3 = np.random.rand(npar, npar, sz)
v3 = v3.transpose(1, 0, 2) + v3

%timeit d2logp(v, v1, msq, v2, msq2, v3, msq3)
%timeit d2logp_(v, v1, msq, v2, msq2, v3, msq3)

5.61 ms ± 339 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.77 ms ± 759 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
sz = 200
npar = 10

m = np.random.rand(sz, sz) 
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz) 
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

mat3 = np.random.rand(npar, npar, sz, sz) 
msq3 = np.einsum('ijkl, ijrl -> ijkr', mat3, mat3)
msq3 = msq3.transpose(1, 0, 2, 3) + msq3

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)
v3 = np.random.rand(npar, npar, sz)
v3 = v3.transpose(1, 0, 2) + v3

%timeit d2logp(v, v1, msq, v2, msq2, v3, msq3)
%timeit d2logp_(v, v1, msq, v2, msq2, v3, msq3)

15.2 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
32.5 ms ± 3.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
sz = 200
npar = 10

m = np.random.rand(sz, sz) 
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz) 
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

mat3 = np.random.rand(npar, npar, sz, sz) 
msq3 = np.einsum('ijkl, ijrl -> ijkr', mat3, mat3)
msq3 = msq3.transpose(1, 0, 2, 3) + msq3

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)
v3 = np.random.rand(npar, npar, sz)
v3 = v3.transpose(1, 0, 2) + v3

%timeit d2logp(v, v1, msq, v2, msq2, v3, msq3)
%timeit d2logp_(v, v1, msq, v2, msq2, v3, msq3)

20.1 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
32.6 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
sz = 400
npar = 1

m = np.random.rand(sz, sz) 
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz) 
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

mat3 = np.random.rand(npar, npar, sz, sz) 
msq3 = np.einsum('ijkl, ijrl -> ijkr', mat3, mat3)
msq3 = msq3.transpose(1, 0, 2, 3) + msq3

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)
v3 = np.random.rand(npar, npar, sz)
v3 = v3.transpose(1, 0, 2) + v3

%timeit d2logp(v, v1, msq, v2, msq2, v3, msq3)
%timeit d2logp_(v, v1, msq, v2, msq2, v3, msq3)

13 ms ± 800 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
16.6 ms ± 415 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Fisher information

In [8]:
sz = 200
npar = 10

m = np.random.rand(sz, sz)
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz)
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)

%timeit fisher(msq, v2, msq2)
%timeit fisher_(msq, v2, msq2)

7.61 ms ± 301 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
16.8 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
sz = 200
npar = 1

m = np.random.rand(sz, sz)
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz)
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)

%timeit fisher(msq, v2, msq2)
%timeit fisher_(msq, v2, msq2)

1.08 ms ± 55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.21 ms ± 85.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
sz = 600
npar = 1

m = np.random.rand(sz, sz)
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz)
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)

%timeit fisher(msq, v2, msq2)
%timeit fisher_(msq, v2, msq2)

27.6 ms ± 2.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
29.5 ms ± 3.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
sz = 20
npar = 1

m = np.random.rand(sz, sz)
msq = m @ m.T

mat2 = np.random.rand(npar, sz, sz)
msq2 = np.einsum('ijk, ilk -> ijl', mat2, mat2)

v = np.random.rand(sz)
v1 = np.random.rand(sz)
v2 = np.random.rand(npar, sz)

%timeit fisher(msq, v2, msq2)
%timeit fisher_(msq, v2, msq2)

60 µs ± 4.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
41.9 µs ± 3.39 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
